# Scraping de dados

Todos dados expostos aqui não são de minha propriedade!!
Eles foram minerados do site [Procure Imóvel](https://procureimovel.com.br/)

In [4]:
import requests as rq
import bs4 as bs4
import time
import json
import pandas as pd
import numpy as np

# Parte 1 

Na Parte 1 as páginas que possuem casas para vender serão coletadas e armazenadas na pasta dados brutos.  
Como resultado teremos vários arquivos html com todas as casas que estão disponíveis no site.

In [2]:
url_casa = "https://procureimovel.com.br/venda/ponta-grossa-pr/casa?page={}"

In [74]:
# Coletando os dados da página principal de casas 
# Primeira coleta

for x in range(1,135):
    urll = url_casa.format(x)
    #print(urll)
    response = rq.get(urll)

    with open('./dados_brutos/casa_{}.html'.format(x), "w+", encoding="utf-8") as output:
        output.write(response.text)
    time.sleep(2)

# Parte 2

Na Parte 2 cada página html será visitada e as informações de título e link serão recolhidas.  
Como saída teremos um dataframe com todas as casas disponíveis para venda e link das mesmas. 

In [12]:
# Segunda coleta

for x in range(1,135):
    with open('./dados_brutos/casa_{}.html'.format(x), 'r+', encoding="utf-8") as inp:
            page_html = inp.read()
            parsed = bs4.BeautifulSoup(page_html)
            tags = parsed.findAll("div", "listing-item")
    for e in tags:
        tags_ = e.find_all('a')
        for f in tags_:
            link = f['href']
            if f.has_attr('title'):
                title = f['title']
                with open('link_and_title.json', 'a+') as output:
                    data = {'link': link, 'title': title}
                    output.write('{}\n'.format(json.dumps(data)))

In [40]:
df = pd.read_json('link_and_title.json', lines=True)

In [41]:
df

,link,title
0,https://procureimovel.com.br/imovel/casa-venda...,"Casa de 164m² à Venda, 3 quartos - Ref. 132860-4"
1,https://procureimovel.com.br/imovel/casa-venda...,Casa com 3 quartos à Venda - Ref. 148460-4
2,https://procureimovel.com.br/imovel/casa-venda...,Casa com 3 quartos à Venda - Ref. 154166-4
3,https://procureimovel.com.br/imovel/casa-venda...,Casa com 2 quartos à Venda - Ref. 134986-4
4,https://procureimovel.com.br/imovel/casa-venda...,"Casa de 70m² à Venda, 2 quartos - Ref. 134520-4"
...,...,...
3989,https://procureimovel.com.br/imovel/casa-venda...,Casa à Venda em Ponta Grossa - Ref. 134613-4
3990,https://procureimovel.com.br/imovel/casa-venda...,"Casa de 990m² à Venda, 4 quartos - Ref. 134592-4"
3991,https://procureimovel.com.br/imovel/casa-venda...,"Casa de 638m² à Venda, 6 quartos - Ref. 142989-4"
3992,https://procureimovel.com.br/imovel/casa-venda...,"Casa de 335m² à Venda, 3 quartos - Ref. 102109-4"


# Parte 3

Cada link do dataframe da Parte 2 será baixado para a pasta dados brutos

In [42]:
lista_de_links = df['link'].unique()

In [43]:
lista_de_links.shape

(3963,)

Serão baixadas 3963 páginas html

In [ ]:
x = 0

inicio = time.time()
for link in lista_de_links:
    #print(link)
    response = rq.get(link)
    with open('./dados_brutos/imovel_{}.html'.format(x), '+w',  encoding="utf-8") as output:
        output.write(response.text)
    #time.sleep(2)
    x = x + 1
fim = time.time()
print("Tempo para processamento de {} páginas: {:.2f}".format(lista_de_links.shape[0], (fim - inicio)))

# Parte 4

Na Parte 4 serão extraídas as informações de cada página html baixada na parte 3  
Como resultado dessa parte teremos um JSON com as informações de todas as casas. 

In [47]:
data = {}
inicio = time.time()
for x in range(0,lista_de_links.shape[0]):
    with open('./dados_brutos/imovel_{}.html'.format(x), 'r+', encoding="utf-8") as inp:
        page_html = inp.read()
        parsed = bs4.BeautifulSoup(page_html)
        tags = parsed.findAll("ul", "property-features margin-top-0")
        if len(tags) == 0:
            continue
        tags = tags[0].findAll('li')
        for e in tags:
            vetor_palavras = 0
            textos = e.get_text()
            vetor_palavras = textos.split(':')
            data[vetor_palavras[0]] = vetor_palavras[-1]
        with open('informacao_casas.json', 'a+') as output:
            output.write('{}\n'.format(json.dumps(data)))
        data.clear()
fim = time.time()
print("Tempo para processamento de {} páginas: {:.2f}".format(lista_de_links.shape[0], (fim - inicio)))

Tempo para processamento de 3963 páginas: 166.59


In [48]:
df = pd.read_json(open("informacao_casas.json", "r", encoding="utf8"), lines=True)